In [2]:
%%capture
!pip install datasets rouge-score nltk transformers sentencepiece

In [1]:
#GPU check
!nvidia-smi

Thu Mar 31 19:55:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
torch.cuda.empty_cache()

In [3]:
# Make sure your version of Transformers is at 4.11.0
# to run the following code correctly:
import datasets
import transformers
import torch

In [4]:
torch_device ='cuda'
#torch_device = 'cpu' (run this line if no gpu attached)

In [5]:
#load original model
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [6]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(torch_device)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [7]:
prefix = "summarize: "

##XSUM

In [8]:
#load test dataset
test_dataset = datasets.load_dataset('xsum', split='test')

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


In [9]:
small_test = test_dataset.select(list(range(2000)))
small_test

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2000
})

In [10]:
# generate summary
batch_size = 32
max_input_length=512
max_target_length=64

def generate_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [11]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/63 [00:00<?, ?ba/s]

In [12]:
results

Dataset({
    features: ['summary', 'id', 'pred'],
    num_rows: 2000
})

In [13]:
metric = datasets.load_metric("rouge")
rouge_score = metric.compute(predictions=results["pred"], references=results["summary"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [14]:
rouge_mean

{'rouge1': 16.525991555512682,
 'rouge2': 2.1437459202762783,
 'rougeL': 13.021018211037788,
 'rougeLsum': 13.019777642857338}

In [22]:
!ls 

bart-base-finetuned-xsum.zip  drive  sample_data


In [15]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/t5-small-finetuned-xsum.zip' '/content'

In [16]:
!unzip t5-small-finetuned-xsum.zip

Archive:  t5-small-finetuned-xsum.zip
   creating: content/t5-small-finetuned-xsum/checkpoint-25506/
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/scaler.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/scheduler.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/tokenizer_config.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/optimizer.pt  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/config.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/pytorch_model.bin  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/tokenizer.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/special_tokens_map.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/rng_state.pth  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/trainer_state.json  
  inflating: content/t5-small-finetuned-xsum/checkpoint-25506/training_args.bin  


In [17]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-xsum/checkpoint-25506/').to(torch_device)

In [18]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['document']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [19]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["document"])

  0%|          | 0/63 [00:00<?, ?ba/s]

In [20]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["summary"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [21]:
rouge_mean

{'rouge1': 16.525991555512682,
 'rouge2': 2.1437459202762783,
 'rougeL': 13.021018211037788,
 'rougeLsum': 13.019777642857338}

In [22]:
finetuned_rouge_mean

{'rouge1': 29.778369926669157,
 'rouge2': 8.677926422408019,
 'rougeL': 23.47989206875079,
 'rougeLsum': 23.470377484890886}

In [23]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['document'])
    print("\nActual Summary: %s" % results[i]['summary'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: Pakistan's telecoms regulator said the ban was no longer necessary because Google, which owns YouTube, had now launched a Pakistan-specific version.
YouTube has denied claims that the authorities can filter content.
Many young Pakistanis have welcomed the lifting of the ban but some activists want details of the deal with Google.
They say there should be greater transparency of the terms agreed between Google and the government.
A Pakistan Telecommunication Authority (PTA) official confirmed to the BBC that all internet service providers had been directed to open access to YouTube.
The Pakistan Telecommunication Company Ltd posted on its Facebook page on Monday: "Welcome Back YouTube".
Pakistan's ministry of information technology said: "Google has provided an online web process through which requests for blocking access of offending material can be made by the PTA to Google directly.
"Google/YouTube will accordingly restrict access to the said offending material for use

##CNN

In [24]:
#load test dataset
test_cnn = datasets.load_dataset('cnn_dailymail', '3.0.0', split='test')

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


In [25]:
small_test = test_cnn.select(list(range(2000)))
small_test

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2000
})

In [26]:
# generate summary
batch_size = 16
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [27]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [28]:
results

Dataset({
    features: ['highlights', 'id', 'pred'],
    num_rows: 2000
})

In [29]:
rouge_score = metric.compute(predictions=results["pred"], references=results["highlights"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [30]:
rouge_mean

{'rouge1': 23.1030101034274,
 'rouge2': 8.986408236979477,
 'rougeL': 18.648127730394737,
 'rougeLsum': 20.977500114970514}

In [31]:
#load fine-tuned model
!cp 'drive/MyDrive/weights/t5-small-finetuned-cnn.zip' '/content'

In [32]:
!unzip t5-small-finetuned-cnn.zip

Archive:  t5-small-finetuned-cnn.zip
   creating: content/t5-small-finetuned-cnn/checkpoint-35890/
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/scaler.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/scheduler.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/tokenizer_config.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/optimizer.pt  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/config.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/pytorch_model.bin  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/tokenizer.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/special_tokens_map.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/rng_state.pth  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/trainer_state.json  
  inflating: content/t5-small-finetuned-cnn/checkpoint-35890/training_args.bin  


In [33]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-cnn/checkpoint-35890/').to(torch_device)

In [34]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['article']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [35]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [36]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["highlights"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [37]:
rouge_mean

{'rouge1': 23.1030101034274,
 'rouge2': 8.986408236979477,
 'rougeL': 18.648127730394737,
 'rougeLsum': 20.977500114970514}

In [38]:
finetuned_rouge_mean

{'rouge1': 24.623718608200644,
 'rouge2': 10.637427641753249,
 'rougeL': 20.302748758901696,
 'rougeLsum': 22.598165213510956}

In [39]:
for i in range(0, 15):
    print("Original Text: %s" % small_test[i]["article"])
    print("\nActual Summary: %s" % results[i]["highlights"])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: (CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood, Best didn't become famous until 1979, when "The Dukes of Hazzard's" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best's Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphras

##Wikihow

In [40]:
#load test dataset
test_wiki = datasets.load_dataset("wikihow", "all", "/content/drive/MyDrive/dataset", split='test')

Using custom data configuration all-6f5101161f12f62f
Reusing dataset wikihow (/root/.cache/huggingface/datasets/wikihow/all-6f5101161f12f62f/1.2.0/5343fc81d685acaa086c9cc19eb8706206cd1f8b315792b04c1d7b92091c305e)


In [41]:
small_test = test_wiki.select(list(range(2000)))
small_test

Dataset({
    features: ['text', 'headline', 'title'],
    num_rows: 2000
})

In [42]:
# generate summary
batch_size = 16
max_input_length=512
max_target_length=128

def generate_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [43]:
results = small_test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [44]:
results

Dataset({
    features: ['headline', 'title', 'pred'],
    num_rows: 2000
})

In [45]:
rouge_score = metric.compute(predictions=results["pred"], references=results["headline"], use_stemmer=True)
# Extract a few results
rouge_mean = {key: value.mid.fmeasure * 100 for key, value in rouge_score.items()}

In [46]:
rouge_mean

{'rouge1': 16.174629778065576,
 'rouge2': 3.349838153623965,
 'rougeL': 12.693453349403388,
 'rougeLsum': 14.893588649729676}

In [ ]:
!ls 

content				   prophetnet-large-uncased-finetuned-wiki.zip
drive				   prophetnet-large-uncased-finetuned-xsum.zip
prophetnet-finetuned-cnn-half.zip  sample_data


In [47]:
#load fine-tuned model
!cp '/content/drive/My Drive/weights/t5-small-finetuned-wiki.zip' '/content'

In [48]:
!unzip t5-small-finetuned-wiki.zip

Archive:  t5-small-finetuned-wiki.zip
   creating: content/t5-small-finetuned-wiki/checkpoint-19657/
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/scaler.pt  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/scheduler.pt  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/tokenizer_config.json  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/optimizer.pt  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/config.json  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/pytorch_model.bin  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/tokenizer.json  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/special_tokens_map.json  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/rng_state.pth  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/trainer_state.json  
  inflating: content/t5-small-finetuned-wiki/checkpoint-19657/training_args.bin  


In [49]:
finetuned_model = T5ForConditionalGeneration.from_pretrained('content/t5-small-finetuned-wiki/checkpoint-19657/').to(torch_device)

In [50]:
# generate summary

def finetuned_summary(batch):
    sentences = [prefix + text for text in batch['text']]
    inputs = tokenizer(sentences, padding="max_length", truncation=True, max_length=max_input_length, return_tensors="pt")
    input_ids = inputs.input_ids.to(torch_device)
    attention_mask = inputs.attention_mask.to(torch_device)

    outputs = finetuned_model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str
    return batch

In [51]:
finetuned_results = small_test.map(finetuned_summary, batched=True, batch_size=batch_size, remove_columns=["text"])

  0%|          | 0/125 [00:00<?, ?ba/s]

In [52]:
finetuned_score = metric.compute(predictions=finetuned_results["pred"], references=finetuned_results["headline"], use_stemmer=True)
# Extract a few results
finetuned_rouge_mean = {key: value.mid.fmeasure * 100 for key, value in finetuned_score.items()}

In [53]:
rouge_mean

{'rouge1': 16.174629778065576,
 'rouge2': 3.349838153623965,
 'rougeL': 12.693453349403388,
 'rougeLsum': 14.893588649729676}

In [54]:
finetuned_rouge_mean

{'rouge1': 24.197973776859396,
 'rouge2': 8.30502818359228,
 'rougeL': 20.336830114247313,
 'rougeLsum': 22.887015549111293}

In [55]:
for i in range(20, 30):
    print("Original Text: %s" % small_test[i]['text'])
    print("\nActual Summary: %s" % results[i]['headline'])
    print("\nOriginal Prediction : %s" % results[i]['pred'])
    print("\nfinetuned Prediction : %s" % finetuned_results[i]["pred"])
    print("=====================================================================\n")

Original Text: In general, dwarf hamsters have thickset bodies, large cheek pouches, and short tails. Make sure it does not have any nasal or eye discharge, nor any other signs of illness. Russian dwarf hamsters are particularly prone to diabetes. A hamster suffering from this condition will drink a lot of water and urinate more frequently than other dwarf hamsters.Be sure to consult your veterinarian if you suspect that your hamster has diabetes.
 A recently weaned or an extremely stressed out dwarf hamster may come down with a disease called “wet tail.” Your hamster experiences diarrhea — the excessive moisture from this causes its tail to become literally wet. Consult a veterinarian for a proper diagnosis and treatment.


Tyzzer’s disease causes diarrhea in young or stressed hamsters. This is a disease that needs veterinarian treatment. Certain antibiotics can cause and exacerbate this condition, so don’t treat your hamster on your own., Like dogs and cats, dwarf hamsters can suffer